# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [520]:
from xml.etree import ElementTree as ET
import pandas as pd
import numpy as np

#Edit XML_FILE to contain your path:
#XML_FILE='./mondial_database.xml'
XML_FILE='C:\SpringboardMay2017\LESSONS\data_wrangling_xml\data_wrangling_xml/data/mondial_database.xml'
document_tree = ET.parse(XML_FILE)

# 10 countries with the lowest infant mortality rates


In [521]:
country_dict={}

In [522]:
# Iterate to find country and infant mortality rate
for child in document_tree.getroot():
    country = child.find('name')
    infant_mortality = child.find('infant_mortality')
    #Capture country and infant mortality rate in a dictionary unless "country" or "infant mortality rate" is missing.
    if country != None:
        if infant_mortality != None:
            country_dict[country.text] = float(infant_mortality.text)
        else:
            country_dict[country.text] = np.NaN
            
#Load country dictionary data from dict to dataframe
country_df = pd.DataFrame.from_dict(country_dict, orient='index')
country_df = country_df.rename(columns={0:'InfantMortalityRate'})
country_df.index.rename('Country',inplace=True)
country_df.sort_values(by='InfantMortalityRate', ascending=True).head(10)


,InfantMortalityRate
Country,
Monaco,1.81
Japan,2.13
Norway,2.48
Bermuda,2.48
Sweden,2.60
Czech Republic,2.63
Hong Kong,2.73
Macao,3.13
Italy,3.31


# 10 cities with the largest population

In [523]:
city_dict= {}
for child in document_tree.iter(tag='city'):
    city = child.find('name')
    population = child.find('population')
    if city != None:
        if population != None:
            city_dict[city.text] = int(child.findall('population')[-1].text)
        else:
            city_dict[city.text] = np.NaN
            
city_df = pd.DataFrame.from_dict(city_dict, orient='index')
city_df = city_df.rename(columns={0:'Population'})
city_df.index.rename('City',inplace=True)
city_df
city_df.sort_values(by='Population', ascending=False).head(10)

,Population
City,
Shanghai,22315474.0
Istanbul,13710512.0
Mumbai,12442373.0
Moskva,11979529.0
Beijing,11716620.0
São Paulo,11152344.0
Tianjin,11090314.0
Guangzhou,11071424.0
Delhi,11034555.0


# 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [524]:
ethnicgrp_dict={}

for child in document_tree.iter(tag='country'):
    country = child.find('name')
    population = child.findall('population')
    ethnic_group = child.findall('ethnicgroup')
    if country != None:
        if population != None:
            temp_pop = int(child.findall('population')[-1].text) #use index [-1] to pick latest population
        else: 
            temp_pop = 0
    for subchild in child.getiterator(tag='ethnicgroup'):  #iterate though country to find 'ethnicgroup' 
        if subchild.text in ethnicgrp_dict:
            ethnicgrp_dict[subchild.text] += int((float(subchild.attrib['percentage'])/100.0) * temp_pop)
        else:
            ethnicgrp_dict[subchild.text] = int((float(subchild.attrib['percentage'])/100.0) * temp_pop)
        
 #Convert to dataframe        
ethnic_df = pd.DataFrame.from_dict(ethnicgrp_dict,orient = 'index')
ethnic_df = ethnic_df.rename(columns={0:'Population'})
ethnic_df.index.rename('Ethnicity',inplace=True)
ethnic_df.sort_values(by='Population',ascending=False).head(10)   

,Population
Ethnicity,
Han Chinese,1245058800
Indo-Aryan,871815583
European,494872201
African,318325104
Dravidian,302713744
Mestizo,157734349
Bengali,146776916
Russian,131856989
Japanese,126534212


# name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [533]:
root = document_tree.getroot()
country_dict = {}

# Creates a dictionary with country code ('car code') and country name
for child in root.iterfind('country'):
#    print(child.attrib['car_code'],child.find('name').text)
    country_dict[child.attrib['car_code']] = child.find('name').text

# a) name and country of longest river:

In [526]:
# Creates blank list
river_list = []

# Loops for rivers' names, lengths, and country codes
for rivers in root.iterfind('river'):
    river_name = rivers.find('name').text
    river_length = rivers.find('length')
    if river_length != None:
        river_length = river_length.text
# Rivers go though multiple countries; codes must be split
        for river_code in rivers.attrib['country'].split():
             river_country_code = country_dict[river_code]
        river_list.append([river_country_code, river_code,
                       river_name, float(river_length)])
    
#for each in river_list:
#    print(each)

In [527]:
# Creates dataframe from list
Country_River_Columns = ['Country', 'Country Code', 'River_Name', 'River_Length']
rivers_df = pd.DataFrame.from_records(river_list, columns=Country_River_Columns)


In [528]:
rivers_df.sort_values('River_Length',ascending=False).head(1)

,Country,Country Code,River_Name,River_Length
174,Peru,PE,Amazonas,6448.0


# b) name and country of largest lake(area)

In [529]:
# Creates blank list
lake_list = []

# Loops for lakess' names, lengths, and country codes
for lakes in root.iterfind('lake'):
    lake_name = lakes.find('name').text
    lake_area = lakes.find('area')
    lake_country = lakes.attrib['country']
    if lake_area != None:
        lake_area = float(lake_area.text)
        for lake_item in lakes.attrib['country'].split():
            lake_list.append([lake_item,country_dict[lake_item],lake_name, lake_area])
            
#for each in lake_list:
#    print(each)
#print(lake_list[0])

In [530]:
lake_df= {}
#Translate list to DataFrame:
lake_df=pd.DataFrame.from_records(lake_list,columns=['Country','Country Code','Lake Name', 'Lake Area'])
lake_df.sort_values('Lake Area', ascending=False).head(1)

,Country,Country Code,Lake Name,Lake Area
68,R,Russia,Caspian Sea,386400.0


# c) airport at highest elevation

In [531]:
airport_list=[]
airport_dict={}
for airport in root.iterfind('airport'):
    airport_country_code = airport.attrib['country']
    airport_name = airport.find('name').text
    airport_elevation = airport.find('elevation').text
    if airport_elevation != None:
        airport_elevation = float(airport_elevation)
        airport_list.append([airport_country_code,country_dict[airport_country_code],airport_name,airport_elevation])

#print(airport_list[0])

In [532]:
airport_dict=pd.DataFrame.from_records(airport_list,columns=['Country Code', 'Country', 'Airport Name', 'Airport Elevation'])
airport_dict.sort_values('Airport Elevation',ascending=False).head(1)


,Country Code,Country,Airport Name,Airport Elevation
80,BOL,Bolivia,El Alto Intl,4063.0
